In [ ]:
import os, sys

import pandas as pd
import numpy as np
import pickle
import gzip

import matplotlib.pyplot as plt

os.environ["CUDA_VISIBLE_DEVICES"]="3" #specify GPU
import keras as K
import tensorflow as tf
from keras import backend
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
def moving_average(a, n=3):
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

Berechnet einfach nur die Durchschnitte der jeweiligen Nachbarwerte eines Arrays. Dadurch wird es zwei Werte kürzer (also fast nichts) berücksichtigte Extremwerte aber wesentlich weniger.

In [ ]:
exec(open(hyperparameter_file).read())

Wir laden die Hyperparamter (hier erstmal nur ein Dummy)

In [ ]:
file = gzip.open(data_file, 'rb')
X_tr, X_val, X_train, X_test, y_tr, y_val, y_train, y_test = pickle.load(file)
file.close()

Wir laden die gesplitteten Daten aus der normalisation Datei

In [ ]:
config = tf.ConfigProto(
         allow_soft_placement=True,
         gpu_options = tf.GPUOptions(allow_growth=True))
set_session(tf.Session(config=config))

Das hier sind Performance Parameter, die je nach System gut sein können (für uns aber wahrscheinlich nicht).

In [ ]:
model = Sequential()
for i in range(len(layers)):
    if i==0:
        model.add(Dense(layers[i], input_shape=(X_tr.shape[1],), activation=act_func,
                        kernel_initializer='he_normal'))
        model.add(Dropout(float(input_dropout)))
    elif i==len(layers)-1:
        model.add(Dense(layers[i], activation='linear', kernel_initializer="he_normal"))
    else:
        model.add(Dense(layers[i], activation=act_func, kernel_initializer="he_normal"))
        model.add(Dropout(float(dropout)))
    model.compile(loss='mean_squared_error', optimizer=K.optimizers.SGD(lr=float(eta), momentum=0.5))

Das hier ist unser Modell, welches wir hier konfigurieren.

In [ ]:
hist = model.fit(X_tr, y_tr, epochs=epochs, shuffle=True, batch_size=64, validation_data=(X_val, y_val))
val_loss = hist.history['val_loss']
model.reset_states()

Jetzt trainieren wir das Modell tatsächlich.

ACHTUNG ! In unserem Code ist nicht das ganze Training drinnen. Die Hyperparameter wurden in mehreren Durchläufen mit verschiedenen Hyperparamtern ausgewahlt, der Code ist nur ein Bsp.

Die Hyperparamter in der Datei sind dennoch korrekt.

In [ ]:
average_over = 15
mov_av = moving_average(np.array(val_loss), average_over)
smooth_val_loss = np.pad(mov_av, int(average_over/2), mode='edge')
epo = np.argmin(smooth_val_loss)

Hier werden die Validation Losses geglättet und dann das Modell der Epoche mit dem geringsten Verlust gewählt.

Das Avg wird über 15 Epochen gebildet und danach die Originallänge des Modells wiedererstellt.

In [ ]:
hist = model.fit(X_train, y_train, epochs=epo, shuffle=True, batch_size=64, validation_data=(X_test, y_test))
test_loss = hist.history['val_loss']

Jetzt trainieren wir das echte Modell mit allen Daten (s. Tabelle welche Teile der Daten in normalisation.iypnb) bis zur errechneten Epoche.

In [ ]:
fig, ax = plt.subplots(figsize=(16,8))
ax.plot(val_loss, label='validation loss')
ax.plot(smooth_val_loss, label='smooth validation loss')
ax.plot(test_loss, label='test loss')
ax.legend()
plt.show()

Jetzt plotten wir die Modellperformance!